# Zaawansowane użycie funkcji

Notebook wprowadza rozszerzone koncepcje pracy z funkcjami w Pythonie.


## Cele
- uporządkować wiedzę o definicjach i wywołaniach funkcji
- omówić funkcje jako obiekty pierwszej klasy oraz domknięcia
- przedstawić generatory, wyrażenia lambda i dekoratory
- zaprezentować kluczowe narzędzia modułu `functools`


## Podstawy: definicja i wywołanie
Funkcja to blok kodu definiowany słowem kluczowym `def`, posiadający parametry, dokumentację oraz wartości domyślne.


In [ ]:
def powitaj(imie: str, powitanie: str = "Cześć") -> str:
    """Zwróć sformatowane powitanie."""
    # Połączenie parametrów w spójną wiadomość
    return f"{powitanie}, {imie}!"

print(powitaj("Ada"))
print(powitaj("Bartek", powitanie="Dzień dobry"))


### Argumenty pozycyjne, słowne i `*args` / `**kwargs`
Elastyczne podpisy funkcji pozwalają budować API o wysokiej ergonomii.


In [ ]:
def opis_uzytkownika(imie, *, rola="uczestnik", **dodatkowe):
    print(f"{imie} pełni rolę: {rola}")
    for klucz, wartosc in dodatkowe.items():
        print(f"- {klucz}: {wartosc}")

opis_uzytkownika("Celina", rola="mentor", projekt="Python Advanced", poziom="średni")


## Funkcje jako obiekty pierwszej klasy
Funkcje można przypisywać do zmiennych, przekazywać jako argumenty i zwracać z innych funkcji. Ta właściwość jest kluczowa dla dalszych technik.


In [ ]:
from typing import Callable

def wykonaj_operacje(a: int, b: int, operacja: Callable[[int, int], int]) -> int:
    # `operacja` to funkcja przekazana z zewnątrz
    return operacja(a, b)

print(wykonaj_operacje(3, 5, lambda x, y: x + y))
print(wykonaj_operacje(10, 4, lambda x, y: x - y))


## Domknięcia (closures)
Domknięcie to funkcja wewnętrzna, która zachowuje odniesienie do zmiennych z otaczającego zakresu, nawet gdy ten zakres już nie istnieje.


In [ ]:
def fabryka_mnoznika(mnoznik: int):
    """Zwróć funkcję mnożącą wejście przez określony mnożnik."""
    def mnoz(x: int) -> int:
        return x * mnoznik  # zmienna `mnoznik` pochodzi z zakresu zewnętrznego
    return mnoz

podwoj = fabryka_mnoznika(2)
potroj = fabryka_mnoznika(3)
print(podwoj(7))
print(potroj(7))


## Funkcje anonimowe (`lambda`)
Wyrażenia lambda pozwalają tworzyć krótkie funkcje w locie. Warto używać ich oszczędnie, gdy pełna definicja `def` byłaby zbyt rozbudowana dla prostej operacji.


In [ ]:
wyniki = [
    ("Ala", 78),
    ("Bartek", 92),
    ("Celina", 85),
]

# Sortowanie po wyniku rosnąco
posortowane = sorted(wyniki, key=lambda wpis: wpis[1])
print(posortowane)

# Filtracja z mapowaniem
prog = 80
zaliczone = list(map(lambda para: para[0], filter(lambda p: p[1] >= prog, wyniki)))
print(zaliczone)


## Generatory i `yield`
Generatory produkują wartości na żądanie, co oszczędza pamięć i pozwala tworzyć złożone potoki danych. Funkcja generatorowa zawiera słowo `yield`.


In [ ]:
def licz_w_przedziale(start: int, stop: int):
    """Zwracaj kolejne liczby, ale tylko te podzielne przez 3 lub 5."""
    for wartosc in range(start, stop):
        if wartosc % 3 == 0 or wartosc % 5 == 0:
            yield wartosc

for liczba in licz_w_przedziale(1, 20):
    print(liczba, end=" ")
print()

# Wyrażenie generatorowe
kwadraty = (n * n for n in range(5))
print(list(kwadraty))


## Moduł `functools`
`functools` dostarcza narzędzi do pracy z funkcjami wysokiego poziomu: memoizacja, częściowa aplikacja, kompozycje i porównania.


In [ ]:
import functools
import operator

@functools.lru_cache(maxsize=128)
def fibonacci(n: int) -> int:
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print([fibonacci(i) for i in range(10)])

# `partial` zamraża część argumentów funkcji
potega2 = functools.partial(pow, 2)
print(potega2(5))

# `reduce` akumuluje wynik
produkt = functools.reduce(operator.mul, [2, 3, 4], 1)
print(produkt)


## Dekoratory
Dekorator to funkcja przyjmująca inną funkcję i zwracająca funkcję opakowującą. Służy do dodawania zachowania bez modyfikacji oryginalnej implementacji.


In [ ]:
import time
import logging
import traceback
from functools import wraps

# --- Konfiguracja logowania ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)-8s | %(name)s | %(funcName)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

In [6]:
# --- Dekorator z pomiarem czasu i logowaniem błędów ---
def loguj_czas(funkcja):
    """Dekorator mierzący czas wykonania i logujący błędy."""
    @wraps(funkcja)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        try:
            wynik = funkcja(*args, **kwargs)
            elapsed = (time.perf_counter() - start) * 1000
            logger.info(f"{funkcja.__name__} wykonana w {elapsed:.2f} ms")
            return wynik
        except Exception as e:
            elapsed = (time.perf_counter() - start) * 1000
            tb = traceback.format_exc()
            logger.error(
                f"Błąd w funkcji {funkcja.__name__} po {elapsed:.2f} ms: {e}\n{tb}"
            )
            raise  # ponownie wyrzucamy wyjątek, by nie maskować błędów
    return wrapper

# --- Przykładowa funkcja ---
@loguj_czas
def policz_do(n: int):
    """Prosty przykład – z celowym błędem dla testu."""
    suma = 0
    for i in range(n):
        suma += i
    return suma

# --- Test ---


@loguj_czas
def funkcja_z_bledem(n: int):
    suma = 0
    for i in range(n):
        suma += i
        if i == n // 2:
            raise ValueError("Blad blad blad")
    return suma
    
policz_do(100_000)

try:
    funkcja_z_bledem(50)
except:
    pass 

2025-10-22 13:11:27 | INFO     | __main__ | wrapper | policz_do wykonana w 4.08 ms
2025-10-22 13:11:27 | ERROR    | __main__ | wrapper | Błąd w funkcji funkcja_z_bledem po 0.01 ms: Blad blad blad
Traceback (most recent call last):
  File "/var/folders/pd/gsgbf0fd4tv7ntlsxt53r0sh0000gn/T/ipykernel_8400/1891676779.py", line 8, in wrapper
    wynik = funkcja(*args, **kwargs)
  File "/var/folders/pd/gsgbf0fd4tv7ntlsxt53r0sh0000gn/T/ipykernel_8400/1891676779.py", line 39, in funkcja_z_bledem
    raise ValueError("Blad blad blad")
ValueError: Blad blad blad



### Dekoratory z argumentami
Dekorator przyjmujący argumenty jest funkcją zwracającą dekorator właściwy. Pozwala parametryzować zachowanie opakowania.


In [ ]:
from functools import wraps

def powtorz(ile_razy: int):
    def dekorator(funkcja):
        @wraps(funkcja)
        def wrapper(*args, **kwargs):
            wynik = None
            for _ in range(ile_razy):
                wynik = funkcja(*args, **kwargs)
            return wynik
        return wrapper
    return dekorator

@powtorz(3)
def powiedz(tekst: str):
    print(tekst)

powiedz("Python jest super!")


## Podsumowanie
- Funkcje w Pythonie to obiekty pierwszej klasy, co umożliwia budowanie domknięć, generatorów i dekoratorów.
- `lambda` i generatory ułatwiają zwięzłe przetwarzanie danych strumieniowych.
- `functools` rozszerza możliwości funkcji o cache, częściową aplikację i operacje redukujące.
- Dekoratory pozwalają wstrzykiwać dodatkowe działania przed/po wywołaniu funkcji, również w sposób parametryzowany.


## Ćwiczenia
1. Zaimplementuj dekorator, który loguje argumenty wejściowe i wynik funkcji, a następnie zastosuj go do funkcji obliczającej średnią.

In [ ]:
# zad 1
import functools

# ----------------------------------------------------
# Dekorator logujący argumenty wejściowe i wynik funkcji
# ----------------------------------------------------
def log_calls(func):
    """Dekorator logujący argumenty i wynik funkcji."""
    @functools.wraps(func)  # zachowuje nazwę i docstring oryginalnej funkcji
    def wrapper(*args, **kwargs):
        print(f"[LOG] Wywołanie funkcji: {func.__name__}")
        print(f"[LOG] Argumenty pozycyjne: {args}")
        print(f"[LOG] Argumenty nazwane: {kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] Wynik: {result}")
        print("-" * 40)
        return result
    return wrapper


# ----------------------------------------------------
# Funkcja obliczająca średnią — udekorowana logowaniem
# ----------------------------------------------------
@log_calls
def average(*numbers):
    """Zwraca średnią z przekazanych liczb."""
    if not numbers:
        raise ValueError("Musisz podać przynajmniej jedną liczbę.")
    return sum(numbers) / len(numbers)


# ----------------------------------------------------
# Przykładowe użycie
# ----------------------------------------------------
average(10, 20, 30)
average(3.5, 4.5, 5.5)
average(1)



2. Stwórz generator produkujący nieskończony ciąg liczb Fibonacciego i wykorzystaj `itertools.islice`, by pobrać pierwsze 10 elementów.

In [ ]:
import itertools

# ----------------------------------------------------
# Generator nieskończonego ciągu liczb Fibonacciego
# ----------------------------------------------------
def fibonacci():
    """Generator zwracający kolejne liczby Fibonacciego."""
    a, b = 0, 1
    while True:          # nieskończona pętla — generator jest "infinite"
        yield a          # zwróć bieżący element
        a, b = b, a + b  # przesuń wartości do przodu


# ----------------------------------------------------
# Pobranie pierwszych 10 elementów za pomocą itertools.islice
# ----------------------------------------------------
first_ten = list(itertools.islice(fibonacci(), 1, 15))

print(first_ten)


3. Napisz funkcję fabrykującą, która przyjmie walidator (funkcję) i zwróci nową funkcję filtrującą listę danych zgodnie z przekazanym walidatorem. Przykladowe funkcje walidujace - to funkcje `is_even` i `is_positive`.


In [ ]:
# ----------------------------------------------------
# Funkcja fabrykująca (factory function)
# ----------------------------------------------------
def make_filter(validator):
    """
    Przyjmuje funkcję walidującą i zwraca nową funkcję,
    która filtruje listę danych zgodnie z przekazanym walidatorem.
    """
    def filter_func(data):
        # Używamy funkcji wbudowanej `filter`, która stosuje validator do każdego elementu
        return list(filter(validator, data))
    return filter_func


# ----------------------------------------------------
# Przykładowe walidatory
# ----------------------------------------------------
def is_even(x):
    """Sprawdza, czy liczba jest parzysta."""
    return x % 2 == 0

def is_positive(x):
    """Sprawdza, czy liczba jest dodatnia."""
    return x > 0


# ----------------------------------------------------
# Tworzenie konkretnych filtrów z pomocą funkcji fabrykującej
# ----------------------------------------------------
even_filter = make_filter(is_even)
positive_filter = make_filter(is_positive)


# ----------------------------------------------------
# Przykładowe dane i wyniki
# ----------------------------------------------------
data = [-3, -2, -1, 0, 1, 2, 3, 4]

print("Parzyste:", even_filter(data))
print("Dodatnie:", positive_filter(data))
